<a href="https://colab.research.google.com/github/yintrigue/melanoma/blob/master/CNN_Trial.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [88]:
from google.colab import files
from google.colab import drive
import os

# optional: mount Google Drive
drive.mount('/content/gdrive')

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [89]:
%cd /content/gdrive/My Drive/w207
!ls

/content/gdrive/My Drive/w207
melanoma


In [90]:
!pip install python-resize-image

In [91]:
import pandas as pd
import numpy as np
from pylab import *
from PIL import Image
from resizeimage import resizeimage
from keras.datasets import cifar10
from keras.models import Sequential
from keras.layers import Dense, Dropout, Conv2D, MaxPool2D, Flatten
from keras.utils import np_utils

In [92]:
train_csv=pd.read_csv('/content/gdrive/My Drive/Kaggle/melanoma/csv/train.csv')
#test_csv=pd.read_csv('/content/gdrive/My Drive/Kaggle/melanoma/csv/test.csv') #ths dataset is not labelled

In [93]:
malignant_df=train_csv[train_csv['benign_malignant']=='malignant'][:500]
benign_df=train_csv[train_csv['benign_malignant']=='benign'][:2000]

malignant_df_dev=train_csv[train_csv['benign_malignant']=='malignant'][500:]
benign_df_dev=train_csv[train_csv['benign_malignant']=='benign'][2000:4000]

In [94]:
train_labels_mal=malignant_df['benign_malignant'].tolist()
image_names_mal=malignant_df['image_name'].tolist()
train_labels_ben=benign_df['benign_malignant'].tolist()
image_names_ben=benign_df['image_name'].tolist()

train_labels_mal_dev=malignant_df_dev['benign_malignant'].tolist()
image_names_mal_dev=malignant_df_dev['image_name'].tolist()
train_labels_ben_dev=benign_df_dev['benign_malignant'].tolist()
image_names_ben_dev=benign_df_dev['image_name'].tolist()

In [95]:
train_data=[]
train_labels=[]
num_mal=0
num_benign=0
for image in image_names_mal:
  try:
    im = Image.open("/content/gdrive/My Drive/Kaggle/melanoma/jpeg_compressed/q_60/train/{}.jpg".format(image))
    im=array(resizeimage.resize('thumbnail', img, [32, 32]))
    train_data.append(im)
    train_labels.append(1)
    num_mal+=1
  except:
    pass

for image in image_names_ben:
    try:
      im = Image.open("/content/gdrive/My Drive/Kaggle/melanoma/jpeg_compressed/q_60/train/{}.jpg".format(image))
      im=array(resizeimage.resize('thumbnail', img, [32, 32]))
      train_data.append(im)
      train_labels.append(0)
      num_benign+=1
    except:
      pass

In [96]:
dev_data=[]
dev_labels=[]
num_mal=0
num_benign=0
for image in image_names_mal_dev:
  try:
    im = Image.open("/content/gdrive/My Drive/Kaggle/melanoma/jpeg_compressed/q_60/train/{}.jpg".format(image))
    im=array(resizeimage.resize('thumbnail', img, [32, 32]))
    dev_data.append(im)
    dev_labels.append(1)
    num_mal+=1
  except:
    pass

for image in image_names_ben_dev:
    try:
      im = Image.open("/content/gdrive/My Drive/Kaggle/melanoma/jpeg_compressed/q_60/train/{}.jpg".format(image))
      im=array(resizeimage.resize('thumbnail', img, [32, 32]))
      dev_data.append(im)
      dev_labels.append(0)
      num_benign+=1
    except:
      pass

In [97]:
train_data=np.array(train_data)
dev_data=np.array(dev_data)
X_train = train_data.reshape(train_data.shape[0], 32, 32, 3)
X_dev = dev_data.reshape(dev_data.shape[0], 32, 32, 3)
X_train = X_train.astype('float32')
X_dev = X_dev.astype('float32')

X_train /= 255
X_dev /= 255

Y_train = np_utils.to_categorical(train_labels, 2)
Y_dev = np_utils.to_categorical(dev_labels, 2)

In [98]:
# building a linear stack of layers with the sequential model
model = Sequential()

# convolutional layer
model.add(Conv2D(50, kernel_size=(3,3), strides=(1,1), padding='same', activation='relu', input_shape=(32, 32, 3)))

# convolutional layer
model.add(Conv2D(75, kernel_size=(3,3), strides=(1,1), padding='same', activation='relu'))
model.add(MaxPool2D(pool_size=(2,2)))
model.add(Dropout(0.25))

model.add(Conv2D(125, kernel_size=(3,3), strides=(1,1), padding='same', activation='relu'))
model.add(MaxPool2D(pool_size=(2,2)))
model.add(Dropout(0.25))

# flatten output of conv
model.add(Flatten())

# hidden layer
model.add(Dense(500, activation='relu'))
model.add(Dropout(0.4))
model.add(Dense(250, activation='relu'))
model.add(Dropout(0.3))
# output layer
model.add(Dense(2, activation='softmax'))

# compiling the sequential model
model.compile(loss='categorical_crossentropy', metrics=['accuracy'], optimizer='adam')

# training the model for 10 epochs
model.fit(X_train, Y_train, batch_size=128, epochs=2, validation_data=(X_dev, Y_dev))


Train on 1223 samples, validate on 1003 samples
Epoch 1/2
1223/1223 [==============================] - 18s 14ms/step - loss: 0.6553 - accuracy: 0.7735 - val_loss: 0.3148 - val_accuracy: 0.9561
Epoch 2/2
1223/1223 [==============================] - 18s 14ms/step - loss: 0.5064 - accuracy: 0.8087 - val_loss: 0.3457 - val_accuracy: 0.9561


In [99]:
model.predict(X_dev)[0]

array([0.7411389 , 0.25886112], dtype=float32)